In [18]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    # Read in and grayscale the image
    gray = grayscale(image)
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 1
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # Define our parameters for Canny and apply
    low_threshold = 1
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(150, imshape[0]),(460, 318), (500, 318), (imshape[1], imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 60     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (y2 != imshape[0] and y1 != imshape[0] and x2!=x1 and y2!=y1):
                m = (y2-y1)/(x2-x1)
                b = y2 - m*x2
                x3 = int((imshape[0]-b)/m)
                x4 = int((320-b)/m)
            else:
                x3 = x2     
                x4 = x1
            cv2.line(line_image,(x4,320),(x3,imshape[0]),(0,255,0),7)    
            
    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 
    
    # Draw the lines on the edge image
    lines_edges = weighted_img(line_image, image, 0.8, 1., 0.)  
    
    return lines_edges


white_output = 'white.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))



[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4





  0%|          | 0/682 [00:00<?, ?it/s]


  0%|          | 3/682 [00:00<00:26, 26.09it/s]


  1%|          | 7/682 [00:00<00:23, 28.74it/s]


  2%|▏         | 11/682 [00:00<00:21, 31.07it/s]


  2%|▏         | 15/682 [00:00<00:20, 32.91it/s]


  3%|▎         | 20/682 [00:00<00:18, 36.18it/s]


  4%|▎         | 24/682 [00:00<00:17, 36.62it/s]


  4%|▍         | 28/682 [00:00<00:20, 31.68it/s]


  5%|▍         | 32/682 [00:00<00:20, 31.80it/s]


  5%|▌         | 36/682 [00:01<00:21, 30.48it/s]


  6%|▌         | 40/682 [00:01<00:22, 29.07it/s]


  6%|▋         | 44/682 [00:01<00:22, 28.76it/s]


  7%|▋         | 47/682 [00:01<00:24, 25.62it/s]


  7%|▋         | 50/682 [00:01<00:28, 21.85it/s]


  8%|▊         | 53/682 [00:01<00:33, 18.54it/s]


  8%|▊         | 56/682 [00:02<00:34, 18.14it/s]


  9%|▊         | 58/682 [00:02<00:35, 17.35it/s]


  9%|▉         | 61/682 [00:02<00:35, 17.32it/s]


  9%|▉         | 63/682 [00:02<00:34, 17.82it/s]


 10%|▉         | 65/682 [00:02<00:34, 1

[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 7.88 s, sys: 492 ms, total: 8.38 s
Wall time: 21.9 s
